# Intro
This is a jupyter notebook for finding distant/unrelated pairs using features generated from the full Pfam-A.hmm instead of the two hmm files used in BiG-SLICE

This code is part of the BiG-SCAPE 2.0 thesis project  

Start by importing these modules